# How the backpropagation algorithm works

First, what’s the algorithm really doing? We’ve developed a picture of the error being backpropagated from the output. But can we go any deeper, and build up more intuition about what is going on when we do all these matrix and vector multiplications? 

The second mystery is how someone could ever have discovered backpropagation in the first place?

## The backpropagation algorithm

The backpropagation equations provide us with a way of computing the gradient of the cost function. Let's explicitly write this out in the form of an algorithm:

1. Input $x$ : Set the corresponding activation $a^{1}$ for the input layer.
2. Feedforward: For each $l=2,3, \ldots, L$ compute $z^{l}=w^{l} a^{l-1}+b^{l}$ and $a^{l}=\sigma\left(z^{l}\right) .$
3. Output error $\delta^{L}$ : Compute the vector $\delta^{L}=\nabla_{a} C \odot \sigma^{\prime}\left(z^{L}\right)$.
4. Backpropagate the error: For each $l=L-1, L-2, \ldots, 2$ compute $\delta^{l}=\left(\left(w^{l+1}\right)^{T} \delta^{l+1}\right) \odot \sigma^{\prime}\left(z^{l}\right)$.
5. Output: The gradient of the cost function is given by $\frac{\partial C}{\partial w_{j k}^{l}}=a_{k}^{l-1} \delta_{j}^{l}$ and $\frac{\partial C}{\partial b_{j}^{l}}=\delta_{j}^{l} .$


Examining the algorithm you can see why it's called backpropagation. We compute the error vectors δl backward, starting from the final layer. It may seem peculiar that we're going through the network backward. But if you think about the proof of backpropagation, the backward movement is a consequence of the fact that the cost is a function of outputs from the network. To understand how the cost varies with earlier weights and biases we need to repeatedly apply the chain rule, working backward through the layers to obtain usable expressions.

**Exercises**

Backpropagation with a single modified neuron Suppose we modify a single neuron in a feedforward network so that the output from the neuron is given by f(∑jwjxj+b), where f is some function other than the sigmoid. How should we modify the backpropagation algorithm in this case?

As I've described it above, the backpropagation algorithm computes the gradient of the cost function for a single training example, $C=C_{x}$. In practice, it's common to combine backpropagation with a learning algorithm such as stochastic gradient descent, in which we compute the gradient for many training examples. In particular, given a mini-batch of $m$ training examples, the following algorithm applies a gradient descent learning step based on that mini-batch:
1. Input a set of training examples
2. For each training example $x$ : Set the corresponding input activation $a^{x, 1}$, and perform the following steps:
   
  * Feedforward: For each $l=2,3, \ldots, L$ compute $z^{x, l}=w^{l} a^{x, l-1}+b^{l}$ and $a^{x, l}=\sigma\left(z^{x, l}\right) .$
  * Output error $\delta^{x, L}$ : Compute the vector $\delta^{x, L}=\nabla_{a} C_{x} \odot \sigma^{\prime}\left(z^{x, L}\right) .$

  * Backpropagate the error: For each
$l=L-1, L-2, \ldots, 2$ compute
$\delta^{x, l}=\left(\left(w^{l+1}\right)^{T} \delta^{x, l+1}\right) \odot \sigma^{\prime}\left(z^{x, l}\right)$.
3. Gradient descent: For each $l=L, L-1, \ldots, 2$ update the weights according to the rule $w^{l} \rightarrow w^{l}-\frac{\eta}{m} \sum_{x} \delta^{x, l}\left(a^{x, l-1}\right)^{T}$, and the biases according to the rule $b^{l} \rightarrow b^{l}-\frac{\eta}{m} \sum_{x} \delta^{x, l}$.

## The code for backpropagation

In [ ]:
import numpy as np

In [ ]:
class Network(object):


   def backprop(self, x, y):
        """Return a tuple "(nabla_b, nabla_w)" representing the
        gradient for the cost function C_x.  "nabla_b" and
        "nabla_w" are layer-by-layer lists of numpy arrays, similar
        to "self.biases" and "self.weights"."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in xrange(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y) 

def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))        


## Backpropagation: the big picture